In [ ]:
import os
import sys
from datetime import datetime
sys.path.insert(0, "../../..")
sys.path.insert(0, "../../../../acquire")

from Acquire.ObjectStore import datetime_to_string
from Acquire.Client import User, Drive, Service, PAR, Authorisation, StorageCreds

from HUGS.Processing import search
from HUGS.Client import Process, Search, Retrieve
from HUGS.Interface import get_map_locations, parse_data, scatter_plot, comparison_plot
from HUGS.Interface import search_emissions, get_emissions_data, plot_emissions

from ipywidgets import (GridspecLayout, GridBox , VBox, HBox, HTML, Layout, Text, 
                        Button, Output, Checkbox, Label)
import ipywidgets as widget

 Enable matplotlib inline plotting

In [ ]:
%matplotlib notebook

# Remove this

In [ ]:
# Autoreload modules before executing code, useful during development
%load_ext autoreload
%autoreload 2

## Create an account on the HUGS platform

To use the HUGS service an account must be created using the Acquire authentication service. Please follow the instructions below to create your account.

In [ ]:
from Acquire.Client import User
from getpass import getpass

Enter a username and password using the cells below

In [ ]:
username = "a_user"
password = "a-strong-password!"

In [ ]:
result = User.register(username=username, password=password, 
                        identity_url="https://google-cloud.acquire-aaai.com/t/identity")

## OTP QR code

Run the cell below to create a QR code to scan using an authenticator app such as AndOTP or Google Authenticator

In [ ]:
result["qrcode"]

## Login

You are now ready to login to the HUGS platform

In [ ]:
from HUGS.Interface import get_login
user, login = get_login()
login


## Searching

In [ ]:
search = Search(service_url=base_url)
start = datetime(1970, 1,1)
end = datetime.now()

search_terms = ["co", "co2"]
search_locations = []
data_type = "CRDS"

search_results = search.search(search_terms=search_terms, locations=search_locations, 
                               data_type=data_type, start_datetime=start, end_datetime=end)


In [ ]:
search_results.keys()

In [ ]:
get_map_locations(search_results)

In [ ]:
to_download = ["bsd_co", "hfd_co", "bsd_co2", "hfd_co2"]

In [ ]:
retrieve = Retrieve(service_url=base_url)
download_keys = {key: search_results[key] for key in to_download}
data = parse_data(retrieve.retrieve(keys=download_keys))

In [ ]:
bsd_data = data["bsd_co"]
scatter_plot(bsd_data)

In [ ]:
comparison_plot(data, ["hfd_co", "hfd_co2"])

## Plot emissions maps

In [ ]:
# Empty search terms, show all data
search_terms = []
results = search_emissions(search_terms=search_terms)
print(results)

In [ ]:
data = get_emissions_data(results[0])

plot_emissions(data)

## TRANSCOM users won't have to upload any data, that'll all be on the platform already

## Search interface - ipywidgets from HUGS.Interface

### Hide all the download keys etc from the user and just give a table of things to plot

### Select the things to plot, return VBox's etc from plotting functions?

In [ ]:
from ipyleaflet import (
    Map,
    Marker, MarkerCluster, TileLayer, ImageOverlay, GeoJSON,
    Polyline, Polygon, Rectangle, Circle, CircleMarker, Popup,
    SplitMapControl, WidgetControl,
    basemaps, basemap_to_tiles
)

from ipywidgets import HTML

In [ ]:
center = [54.2361, -4.548]
zoom = 5
m = Map(center=center, zoom=zoom)
mark_bsd = Marker(location=(54.942544, -1.369204))
mark_bsd.popup = HTML(value="Bilsdale")

mark_mhd = Marker(location=(53.20,-9.54))
mark_mhd.popup = HTML(value="Macehead")

mark_tac = Marker(location=(52.511, 1.155003))
mark_tac.popup = HTML(value="Tacolneston")

m += mark_bsd
m += mark_mhd
m += mark_tac

display(m)